In [42]:
import os
import cv2
import numpy as np
import pandas as pd
from glob import glob
from PIL import Image
import matplotlib.pyplot as plt
%cd H:\Python\HAM1000
%ls

H:\Python\HAM1000
 Volume in drive H is SSD
 Volume Serial Number is 1E82-13E0

 Directory of H:\Python\HAM1000

05/07/2024  03:51 AM    <DIR>          .
05/07/2024  03:06 AM    <DIR>          ..
05/07/2024  03:07 AM    <DIR>          HAM10000_images
10/06/2019  04:59 PM           563,277 HAM10000_metadata.csv
05/07/2024  03:51 AM    <DIR>          HAM10000_npy
10/06/2019  05:03 PM        30,807,979 hmnist_28_28_L.csv
10/06/2019  05:03 PM        91,820,383 hmnist_28_28_RGB.csv
10/06/2019  05:03 PM         2,537,778 hmnist_8_8_L.csv
10/06/2019  05:03 PM         7,524,968 hmnist_8_8_RGB.csv
               5 File(s)    133,254,385 bytes
               4 Dir(s)  57,094,549,504 bytes free


In [48]:
import os

dataset_path = r'H:\Python\HAM1000\HAM10000_images'
import os

dataset_path = r'H:\Python\HAM1000\HAM10000_images'
files = os.listdir(dataset_path)
dataset_files = {os.path.splitext(os.path.basename(x))[0]: os.path.join(dataset_path, x)
                 for x in files if x.lower().endswith('.jpg')}
print(len(dataset_files))
lesion_type_dict = {
    'nv': 'Melanocytic nevi (nv)',
    'mel': 'Melanoma (mel)',
    'bkl': 'Benign keratosis-like lesions (bkl)',
    'bcc': 'Basal cell carcinoma (bcc)',
    'akiec': 'Actinic keratoses (akiec)',
    'vasc': 'Vascular lesions (vasc)',
    'df': 'Dermatofibroma (df)'
}

label_mapping = {
    0: 'nv',
    1: 'mel',
    2: 'bkl',
    3: 'bcc',
    4: 'akiec',
    5: 'vasc',
    6: 'df'
}

reverse_label_mapping = dict((value, key) for key, value in label_mapping.items())
data = pd.read_csv(os.path.join('/Python/HAM1000', 'HAM10000_metadata.csv'))
data.head()

10015


,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [49]:
data.describe(exclude=[np.number])

print("Null found: ", end="")
print(data.isnull().any().sum())  
data['age'].fillna(value=int(data['age'].mean()), inplace=True)
data['age'] = data['age'].astype('int32')
data['cell_type'] = data['dx'].map(lesion_type_dict.get)
data['path'] = data['image_id'].map(dataset_files.get)
data.head()


Null found: 1


,lesion_id,image_id,dx,dx_type,age,sex,localization,cell_type,path
0,HAM_0000118,ISIC_0027419,bkl,histo,80,male,scalp,Benign keratosis-like lesions (bkl),H:\Python\HAM1000\HAM10000_images\ISIC_0027419...
1,HAM_0000118,ISIC_0025030,bkl,histo,80,male,scalp,Benign keratosis-like lesions (bkl),H:\Python\HAM1000\HAM10000_images\ISIC_0025030...
2,HAM_0002730,ISIC_0026769,bkl,histo,80,male,scalp,Benign keratosis-like lesions (bkl),H:\Python\HAM1000\HAM10000_images\ISIC_0026769...
3,HAM_0002730,ISIC_0025661,bkl,histo,80,male,scalp,Benign keratosis-like lesions (bkl),H:\Python\HAM1000\HAM10000_images\ISIC_0025661...
4,HAM_0001466,ISIC_0031633,bkl,histo,75,male,ear,Benign keratosis-like lesions (bkl),H:\Python\HAM1000\HAM10000_images\ISIC_0031633...


In [50]:
data['image_pixel'] = data['path'].map(lambda x: np.asarray(Image.open(x).resize((32,32))))
data.head(5)

,lesion_id,image_id,dx,dx_type,age,sex,localization,cell_type,path,image_pixel
0,HAM_0000118,ISIC_0027419,bkl,histo,80,male,scalp,Benign keratosis-like lesions (bkl),H:\Python\HAM1000\HAM10000_images\ISIC_0027419...,"[[[191, 152, 192], [194, 154, 193], [196, 152,..."
1,HAM_0000118,ISIC_0025030,bkl,histo,80,male,scalp,Benign keratosis-like lesions (bkl),H:\Python\HAM1000\HAM10000_images\ISIC_0025030...,"[[[24, 14, 29], [56, 39, 64], [106, 80, 112], ..."
2,HAM_0002730,ISIC_0026769,bkl,histo,80,male,scalp,Benign keratosis-like lesions (bkl),H:\Python\HAM1000\HAM10000_images\ISIC_0026769...,"[[[190, 136, 150], [199, 145, 161], [200, 141,..."
3,HAM_0002730,ISIC_0025661,bkl,histo,80,male,scalp,Benign keratosis-like lesions (bkl),H:\Python\HAM1000\HAM10000_images\ISIC_0025661...,"[[[35, 18, 28], [83, 51, 64], [128, 88, 103], ..."
4,HAM_0001466,ISIC_0031633,bkl,histo,75,male,ear,Benign keratosis-like lesions (bkl),H:\Python\HAM1000\HAM10000_images\ISIC_0031633...,"[[[155, 110, 135], [188, 139, 168], [210, 157,..."


In [51]:
data['label'] = data['dx'].map(reverse_label_mapping.get)
data = data.sort_values('label')
data = data.reset_index()

In [52]:
X_orig = data['image_pixel'].to_numpy()
X_orig = np.stack(X_orig, axis=0)
Y_orig = np.array(data.iloc[:, -1:])
print(X_orig.shape)
print(Y_orig.shape)

(10015, 32, 32, 3)
(10015, 1)


# Train Split

In [53]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def prepare_for_train_test_validation(X, Y, test_size=0.2, validation_size=0.2, random_state=1, save_path=None):
    # Splitting the data into train, test, and validation sets
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=random_state)
    X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=validation_size, random_state=random_state)
    
    # Data augmentation for training set
    train_datagen = ImageDataGenerator(rescale = 1./255,
                                  rotation_range = 10,
                                  width_shift_range = 0.2,
                                  height_shift_range = 0.2,
                                  shear_range = 0.2,
                                  horizontal_flip = True,
                                  vertical_flip = True,
                                  fill_mode = 'nearest')
    train_datagen.fit(X_train)
    
    # Data normalization for test and validation sets
    test_datagen = ImageDataGenerator(rescale = 1./255)
    test_datagen.fit(X_test)
    val_datagen = ImageDataGenerator(rescale = 1./255)
    val_datagen.fit(X_val)
    
    # Optionally save the splits as .npy files
    if save_path:
        np.save(save_path + '/train_image.npy', X_train)
        np.save(save_path + '/train_label.npy', Y_train)
        np.save(save_path + '/test_image.npy', X_test)
        np.save(save_path + '/test_label.npy', Y_test)
        np.save(save_path + '/val_image.npy', X_val)
        np.save(save_path + '/val_label.npy', Y_val)
    
    return X_train, X_test, X_val, Y_train, Y_test, Y_val


X_train, X_test, X_val, Y_train, Y_test, Y_val = prepare_for_train_test_validation(X_orig, Y_orig, save_path=r'H:\Python\HAM1000\HAM10000_npy')
